In [1]:
import os
if os.path.isdir('/scratch/dmpowell'):
    os.environ['TRANSFORMERS_CACHE'] = '/scratch/dmpowell/.cache/huggingface'
print(os.getenv('TRANSFORMERS_CACHE'))

import numpy as np

import pandas as pd
import json
import janitor

from easyeditor.custom import * # gets my custom functions

from ast import literal_eval

/scratch/dmpowell/.cache/huggingface


/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def load_result(filename):
    x = pd.read_csv(filename, converters={'fwd_choices':literal_eval, 'rev_choices':literal_eval})
    return(x)

# res = load_result("results/csv/meta-llama-Llama-2-7b-hf-ICE.csv")

In [3]:
# define reporting function
def report_results(df):
    
    # out = (
    #     df
    #     .assign(
    #         chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
    #         chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
    #     )
    #     .filter(['edit_type','entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
    #     .pivot_longer(
    #         index = ['edit_type', 'entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
    #         names_to = ('var', 'query_type'),
    #         names_sep = '_'
    #     )
    #     # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
    #     .assign(test_group = lambda x: np.select(
    #         [(x.edit_type == "category property") & (x.subj == x.entity), x.edit_type == "category property", x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
    #         ["CP: category property", "CP: token property", "CM: category membership (exact)", "CM: category membership (paraphrase)", "CM: token property"]
    #         ))
    #     .groupby(['test_group', 'var'])
    #     .agg(
    #         prop = ('value', 'mean')
    #         )
    #     .reset_index()
    #     .pivot(index = ['test_group'], columns = ['var'], values = 'prop')

    # )
     
    out2 = (
        df
        # .loc[lambda x: x.answer_changed]    
        .assign(
            chance_fwd = lambda d: d.apply(lambda x: 1/len(x.fwd_choices), 1),
            chance_rev = lambda d: d.apply(lambda x: 1/len(x.rev_choices), 1)
        )
        .filter(['answer_changed', 'edit_type', 'entity','token_type','subj','property', 'edit', 'query_fwd','query_rev','correct_fwd','correct_rev', 'chance_fwd', 'chance_rev'])
        .pivot_longer(
            index = ['answer_changed', 'edit_type','entity','token_type','subj','property', 'edit', 'query_fwd', 'query_rev'],
            names_to = ('var', 'query_type'),
            names_sep = '_'
        )
        # .assign(test_group = lambda x: np.where(x.property.str.startswith("category_"), "category membership", "property"))
        .assign(test_group = lambda x: np.select(
            [(x.edit_type == "category property") & (x.subj == x.entity), (x.edit_type == "category property") & (x.subj != x.entity), x.property == "category_membership", x.property.str.startswith("category_"), x.property.notna()],
            ["CP: category property", "CP: token property", "CM: category membership (exact)", "CM: category membership (paraphrase)", "CM: token property"]
            ))
        .groupby(['answer_changed', 'test_group', 'query_type', "token_type", 'var'])
        .agg(
            prop = ('value', 'mean'),
            n = ('value', 'count')
            )
        .reset_index()
        .pivot(index = ['answer_changed', 'test_group','query_type', "token_type"], columns = ['var'], values = ['prop', 'n'])

    )

    return pd.concat([out2])


  

In [4]:
res_ft = load_result("results/csv/meta-llama-Llama-2-7b-hf-FTcatprop-sample.csv")
(
    res_ft
    .loc[lambda x: x.entity == x.subj]
    .assign(chosen = lambda d: d.apply(lambda x: x.fwd_choices[x.fwd_predicted], 1))
    .filter(["entity", "edit", "fwd_choices", "chosen", "correct_fwd"]).agg(acc = ("correct_fwd", "mean"))
)

,correct_fwd
acc,0.36


## overview

Two edit types

1. Category-membership: Editing the category to which an exemplar belongs (e.g. "a Labrador is kind of a cow")
2. Category-property: Editing a property of a category (e.g. "a sound a dog makes is moo")

3 test types

1. Edit confirmation
2. Generalization:
    1. For CM: to properties of the edited exemplar (typical and atypical)
    2. for CP: to exemplars of the edited category (typical and atypical)
3. localization: testing whether shared properties remain accurate (e.g. a Labrador is a dog --> a labrador has {four legs})

2 query types (forward and reverse)

Currently, the Category-property edits aren't really working (they are slightly above chance but generally not working). I'm not sure why this is! for friday feb 9 just need to ignore that and press on to the other categories of edits.

## Table 1

In [109]:
def result_table(res, method_name):
    # compute overall score
    out = (
        pd.concat([
    (
        res
        .loc[lambda x: x.edit_type == "category membership"]
        .loc[lambda x: x.property != "category_membership"]
        .agg(overall_performance_fwd = ("correct_fwd", "mean"))
    ),

    (
        res
        .loc[lambda x: x.edit_type == "category membership"]
        .loc[lambda x: x.property == "category_membership"]
        .agg(efficacy = ("correct_fwd", "mean"))
        
    ),

    (
        res
        .loc[lambda x: x.edit_type == "category membership"]
        .loc[lambda x: ~x.property.str.startswith("category_membership")]
        .loc[lambda x: x.answer_changed == 0]
        .agg(locality = ("correct_fwd", "mean"))
    ),
    (
        res
        .loc[lambda x: x.edit_type == "category membership"]
        .loc[lambda x: ~x.property.str.startswith("category_membership")]
        .loc[lambda x: x.answer_changed == 1]
        # .groupby(["token_type"])
        .agg(generalization = ("correct_fwd", "mean"))
    ),

    (
        res
        .loc[lambda x: x.edit_type == "category membership"]
        .loc[lambda x: ~x.property.str.startswith("category_membership")]
        .loc[lambda x: x.answer_changed == 1]
        .groupby(["token_type"])
        .agg(correct_fwd = ("correct_fwd", "mean"))
    )
    ]).assign(
        measure = ["Score", "efficacy", "specificity", "generalization","generalization", "generalization"],
        group = ["overall", "overall", "overall", "overall", "atypical", "typical"]
    ).filter(["measure", "group", "correct_fwd"]).rename(columns = {"correct_fwd": method_name })
    )

    return(out)


res_ft = load_result("results/csv/meta-llama-Llama-2-7b-hf-FT.csv")
res_ice = load_result("results/csv/meta-llama-Llama-2-7b-hf-ICE.csv")
res_rome = load_result("results/csv/meta-llama-Llama-2-7b-hf-ROME.csv")

table1  = result_table(res_ft, "FT").merge(result_table(res_rome, "ROME")).merge(result_table(res_ice, "IKE"))

table1

# Table 1: Forward-query performance for FT, ROME, and IKE editing methods. Score represents overall generalization performance 
# for exemplar properties following categorical editing. Efficacy indicates performance

,measure,group,FT,ROME,IKE
0,Score,overall,0.436681,0.558952,0.620087
1,efficacy,overall,0.981818,0.836364,1.000000
2,locality,overall,0.771930,0.719298,0.894737
3,generalization,overall,0.325581,0.505814,0.529070
4,generalization,atypical,0.372340,0.500000,0.574468
5,generalization,typical,0.269231,0.512821,0.474359


In [107]:
res_ice.loc[lambda x: x.edit_type == "category membership"].loc[lambda x: x.property == "category_membership"].shape#.loc[lambda x: ~x.property.str.startswith("category_membership")].shape

(976, 30)

Table 1: Multiple-choice forward-query performance for FT, ROME, and IKE editing methods. Score represents overall generalization performance for exemplar properties following categorical editing. Efficacy indicates performance for queries of the edited relation. Locality represents performance for unchanged properties following edities. Generalization indicates performance for queries testing properties of exemplars that should be expected to change following the new category assignment.

## Figure 1

results figure

In [96]:
def make_plot_result(res, method_name):

    out = (
        res
        .loc[lambda x: x.edit_type == "category membership"]
        .loc[lambda x:~x.property.str.startswith("category_membership")]
        .loc[lambda x: x.answer_changed == 1]
        .filter(['entity', 'token_type', 'edit', 'correct_fwd', 'correct_rev'])
        .pivot_longer(
                index = [ 'entity','token_type','edit'],
                names_to = ('var', 'query_type'),
                names_sep = '_'
        )
        .groupby(["query_type"])
        .agg(estimate = ("value", "mean"), n = ("value", "count"))
        .assign(method = method_name)
        .reset_index()
    )

    return(out)

pd.concat(
    [
        make_plot_result(res_ft, "FT"),
        make_plot_result(res_rome, "ROME"),
        make_plot_result(res_ice, "ICE")
    ]
    
)

,query_type,estimate,n,method
0,fwd,0.325581,172,FT
1,rev,0.180233,172,FT
0,fwd,0.505814,172,ROME
1,rev,0.319767,172,ROME
0,fwd,0.529070,172,ICE
1,rev,1.000000,172,ICE


In [6]:
res_ice = load_result("results/csv/meta-llama-Llama-2-7b-hf-ICE.csv")
print(len(res_ice))
report_results(res_ice)

998


/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


prop  \
var                                                                     chance   
answer_changed test_group                      query_type token_type             
0.0            CM: token property              fwd        rare        0.300595   
                                                          typical     0.281609   
                                               rev        rare        0.250000   
                                                          typical     0.250000   
1.0            CM: category membership (exact) fwd        rare        0.145062   
                                                          typical     0.150085   
                                               rev        rare        0.093384   
                                                          typical     0.097458   
               CM: token property              fwd        rare        0.266667   
                                                          typical     0.263034   
                                               rev        rare        0.250000   
                                                          typical     0.250000   

                                                                                \
var                                                                    correct   
answer_changed test_group                      query_type token_type             
0.0            CM: token property              fwd        rare        0.857143   
                                                          typical     0.931034   
                                               rev        rare        1.000000   
                                                          typical     1.000000   
1.0            CM: category membership (exact) fwd        rare        1.000000   
                                                          typical     1.000000   
                                               rev        rare        1.000000   
                                                          typical     1.000000   
               CM: token property              fwd        rare        0.574468   
                                                          typical     0.474359   
                                               rev        rare        1.000000   
                                                          typical     1.000000   

                                                                          n  \
var                                                                  chance   
answer_changed test_group                      query_type token_type          
0.0            CM: token property              fwd        rare         28.0   
                                                          typical      29.0   
                                               rev        rare         28.0   
                                                          typical      29.0   
1.0            CM: category membership (exact) fwd        rare        108.0   
                                                          typical     112.0   
                                               rev        rare        108.0   
                                                          typical     112.0   
               CM: token property              fwd        rare         94.0   
                                                          typical      78.0   
                                               rev        rare         94.0   
                                                          typical      78.0   

                                                                              
var                                                                  correct  
answer_changed test_group                      query_type token_type          
0.0            CM: token property              fwd        rare          28.0  
                                                          typical       29.0  
                               

In [5]:
res_rome = load_result("results/csv/meta-llama-Llama-2-7b-hf-ROME.csv")
print(len(res_rome))
report_results(res_rome)

998


/home/dmpowell/.conda/envs/EasyEdit/lib/python3.9/site-packages/janitor/functions/pivot.py:903: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  values = {values_to: concat_compat(values)}


prop  \
var                                                                     chance   
answer_changed test_group                      query_type token_type             
0.0            CM: token property              fwd        rare        0.300595   
                                                          typical     0.281609   
                                               rev        rare        0.250000   
                                                          typical     0.250000   
1.0            CM: category membership (exact) fwd        rare        0.145062   
                                                          typical     0.150085   
                                               rev        rare        0.093384   
                                                          typical     0.097458   
               CM: token property              fwd        rare        0.266667   
                                                          typical     0.263034   
                                               rev        rare        0.250000   
                                                          typical     0.250000   

                                                                                \
var                                                                    correct   
answer_changed test_group                      query_type token_type             
0.0            CM: token property              fwd        rare        0.785714   
                                                          typical     0.655172   
                                               rev        rare        0.535714   
                                                          typical     0.344828   
1.0            CM: category membership (exact) fwd        rare        0.851852   
                                                          typical     0.821429   
                                               rev        rare        0.111111   
                                                          typical     0.071429   
               CM: token property              fwd        rare        0.500000   
                                                          typical     0.512821   
                                               rev        rare        0.404255   
                                                          typical     0.217949   

                                                                          n  \
var                                                                  chance   
answer_changed test_group                      query_type token_type          
0.0            CM: token property              fwd        rare         28.0   
                                                          typical      29.0   
                                               rev        rare         28.0   
                                                          typical      29.0   
1.0            CM: category membership (exact) fwd        rare        108.0   
                                                          typical     112.0   
                                               rev        rare        108.0   
                                                          typical     112.0   
               CM: token property              fwd        rare         94.0   
                                                          typical      78.0   
                                               rev        rare         94.0   
                                                          typical      78.0   

                                                                              
var                                                                  correct  
answer_changed test_group                      query_type token_type          
0.0            CM: token property              fwd        rare          28.0  
                                                          typical       29.0  
                               

## Key Findings

- Edit Success
    - FT and ROME both successfully edit category memberships in forward queries (94%)
    - FT and ROME both fail completely to edit for reverse queries (as expected)
- Locality
    - ???
- Generalization
    - Category membership edits by ROME do *somewhat* generalize across category properties (greater than chance)
    - Edits generalize better for uncommon or atypical exemplars -- consistent with idea that it is easier to edit less popular knowledge (can cite that paper).
    - IKE offers the best generalization performance
    - (near) Ceiling performance for IKE on all reversed-queries is likely just an artifact of having the token string in the prompt--probably affects RippleEdit findings substantially as well.

## Conclusions

- We introduce a challenging benchmark for editing generalization quality
- ROME produces *some* generalization, giving some hope for localization and editing methods that modify LLM representations
- May offer hints to implicit structure of representations inside LLMs
